In [86]:
!pip3 install requests bs4 openpyxl plotly_express
!pip3 install -U kaleido

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# ---------------------------------------- Coleta de Dados (Raspagem) ---------------------------------------

from selenium import webdriver                  # Simula o uso do navegador
from selenium.webdriver.common.by import By     # Permite encontrar informações dentro de sites
import openpyxl                 # Permite criar e manipular planilhas do excel
import pandas as pd             # Abrir e manipular arquivos em excel

# ---------------------------------------- Análise de Dados (Gráfico) ---------------------------------------

import plotly.express as px     # Gerar gráficos interativos
import plotly                   # Usado para baixar o gráfico

In [88]:
# URL do site que será feito o scraping
url = 'https://www.kabum.com.br/computadores/monitores/monitor-gamer'

# Cria uma planilha do excel para armazenar os dados (database.xlsx)
planilha = openpyxl.Workbook()

# Cria uma página na planilha (monitores)
planilha.create_sheet('monitores')

# Seleciona a página criada e insere os títulos das colunas
sheet_monitores = planilha['monitores']
sheet_monitores['A1'].value = 'Nome do Monitor'
sheet_monitores['B1'].value = 'Preço'

# Salvando a planilha
planilha.save('database.xlsx')

In [89]:
# Função para coletar o link da proxima pagina
def next_page(driver):
    # Verifica se existe o elemento que indica o fim das páginas
    if not driver.find_elements(By.XPATH,"//li[@class='next disabled']"):
        # Retorna o conteudo do elemento que indica o link da próxima página
        return driver.find_element(By.XPATH,"//link[@rel='next']").get_attribute('href')
    else:
        return None


In [90]:
# inicializando o webdriver (navegador que será usado) - Chrome
driver = webdriver.Chrome()

# Abrindo a planilha
wb = openpyxl.load_workbook('database.xlsx')

# Selecionando a página da planilha
sheet_produtos = wb['monitores']

while True:
    
    # --------------------------------------- Abrindo a Página Atual ---------------------------------------
    
    driver.get(url) # Abre a página no navegador

    # ---------------------------------------- Coletando os Dados --------------------------------------------

    # Coletando os dados utilizando o xpath
    titles = driver.find_elements(By.XPATH,"//span[@class='sc-d79c9c3f-0 nlmfp sc-93fa31de-16 bBOYrL nameCard']")
    promotional_prices = driver.find_elements(By.XPATH,"//span[@class='sc-6889e656-2 bYcXfg priceCard']")

    # Inserindo os dados na planilha (somente tuplas de nome, preço com valors diferentes de null para manter consistência)
    for title, price in zip(titles, promotional_prices):
        sheet_produtos.append([title.text, price.text])
    
    # ------------------------------------------ Próxima Página -----------------------------------------------

    # Recebe o link da proxima pagina
    url = next_page(driver)

    if url == None:
        break

# Salvando a planilha
wb.save('database.xlsx')

# Fechando o navegador (webdriver)
driver.quit()

In [93]:
# Abrindo a planilha
wb = openpyxl.load_workbook('database.xlsx')

# Acessando a página 'produtos'
sheet_produtos = wb['monitores']

# ----------------------------------------------- FORMATANDO OS DADOS (PREÇO) ----------------------------------------------- #
# Excluindo excesso de informação nos preços
for cell in sheet_produtos['B']:
    cell.value = cell.value.replace('R$ ', '')  # Excluindo o 'R$ ' pois atrapalha a conversão para float
    cell.value = cell.value.replace('.', '')    # Excluindo o '.' pois também atrapalha a conversão para float
    cell.value = cell.value.replace(',', '.')   # Trocando a vírgula por ponto para evitar problemas com string e float

# Convertendo os preços para float (com exceção do cabeçalho)
for cell in sheet_produtos['B'][1:]:
  cell.value = float(cell.value)

# Salvando a planilha
wb.save('database.xlsx')

In [5]:
# ----------------------------------------------- ABRINDO A PLANILHA ----------------------------------------------- #

# Abrindo a planilha
wb = openpyxl.load_workbook('database.xlsx')

# Acessando a página 'produtos'
sheet_produtos = wb['monitores']

# ---------------------------------------- ORDENANDO OS DADOS PARA O GRÁFICO ---------------------------------------- #

# Criando uma lista para armazenar os produtos (nome, preço)
produtos = []

# Inserindo os produtos na lista (OBS.: zip() é uma função que permite iterar sobre duas listas ao mesmo tempo)
for title, price in zip(sheet_produtos['A'][1:], sheet_produtos['B'][1:]):
  produtos.append([title.value, price.value])

# Ordenando a lista de produtos pelo preço (crescente)
produtos.sort(key=lambda produto: produto[1]) # key = lambda é um parâmetro que permite ordenar a lista por um valor específico, nesse caso o preço
 
# exibindo a lista de produtos com quebra de linha
#for produto in produtos:
# print(produto[0], produto[1], '\n')

# -------------------------------------------------- GERANDO O GRÁFICO -------------------------------------------------- #

# Criando um dataframe com os dados da lista de produtos
df = pd.DataFrame(produtos, columns=['Produto', 'Preço Promocional'])

# Gerando o gráfico de linhas
fig = px.line(
    df, 
    x = df.index,
    y = 'Preço Promocional',
    labels={'x': 'PRODUTO', 'Preço Promocional': 'PREÇO'},
    hover_data=['Produto'],
    markers=True,
    #color = 'Produto',
    title = 'BENCHMARK DE PREÇOS DE MONITORES GAMER - KABUM',
)

fig.show()

# Baixando o gráfico
plotly.offline.plot(fig)

'temp-plot.html'